Data directories

In [1]:
dataset_dir = '../tcga'
expression_dir = dataset_dir + '/mrna_expression_data/'
mutation_dir = dataset_dir + '/mrna_mutation_data/'

Mutation annotation files

In [46]:
import os
import pandas as pd

In [7]:
mutation_entity_ids = []
for case_id in os.listdir(mutation_dir):
    try:
        annotations_df = pd.read_csv(mutation_dir+case_id+'/annotations.txt', sep='\t')
        mutation_entity_ids.append(annotations_df['entity_id'].iloc[0])
    except:
        print(f'Missing annotation: {case_id}')
    

print(len(mutation_entity_ids))

Missing annotation: 1e171ca7-9c44-4dc9-8908-b35b40e3464e
Missing annotation: 23c0af0c-102c-40e6-8e93-7ec13bc032b9
Missing annotation: 2e9ac636-1882-4d71-b672-c057d2730f03
Missing annotation: 56d15f8e-3788-4715-9ec8-853a107679a4
Missing annotation: 7a7dcf94-b3d7-4516-9736-db8fdb7b451a
Missing annotation: 803decea-3a3f-42bf-8122-328bc32aa850
Missing annotation: c1a21a51-b8b7-453d-932e-2eacbaed8c2d
Missing annotation: cb5fd63b-da6d-4016-b150-2a0682d00db2
Missing annotation: e0e31082-4736-4917-a72b-9bf455239984
Missing annotation: f3508f1c-34c8-4e72-87b3-0163273b0393
Missing annotation: f90b3b34-16b3-435c-bbf1-82a1450ec7f0
981


In [23]:
import requests

def lookup_file_metadata(file_uuid):
    url = f"https://api.gdc.cancer.gov/files/{file_uuid}?expand=cases.samples.portions.analytes.aliquots"
    response = requests.get(url)
    return response.json()

# Example
print('Entity id:', mutation_entity_ids[0])
mutation_metadata = lookup_file_metadata(mutation_entity_ids[0])
print(mutation_metadata)

Entity id: 002c911e-0d8a-4bce-84af-341a32cae334
{'data': {'data_format': 'MAF', 'cases': [{'samples': [{'portions': [{'analytes': [{'aliquots': [{'aliquot_quantity': 2.08, 'amount': None, 'aliquot_id': 'b3568259-c63c-4eb1-bbc7-af711ddd33db', 'source_center': '23', 'analyte_type': None, 'updated_datetime': '2022-04-29T07:04:19.361005-05:00', 'submitter_id': 'TCGA-A1-A0SO-01A-22D-A099-09', 'concentration': 0.08, 'state': 'released', 'aliquot_volume': 26.0, 'analyte_type_id': None, 'created_datetime': None}]}]}]}, {'portions': [{'analytes': [{'aliquots': [{'aliquot_quantity': 1.82, 'amount': None, 'aliquot_id': '17ba8cdb-e35b-4496-a787-d1a7ee7d4a1e', 'source_center': '23', 'analyte_type': None, 'updated_datetime': '2022-04-29T07:05:03.186372-05:00', 'submitter_id': 'TCGA-A1-A0SO-10A-03D-A099-09', 'concentration': 0.07, 'state': 'released', 'aliquot_volume': 26.0, 'analyte_type_id': None, 'created_datetime': None}]}]}]}]}], 'access': 'open', 'file_name': '0b515b8a-b1ab-4508-8185-952fb73d9b

In [24]:
print('UUID:', expression_case_ids[0])
expression_metadata = lookup_file_metadata(expression_case_ids[0])
#print(metadata)
print(expression_metadata['data']['cases'][0])
#metadata['data']['cases'][0]['case_id']
# mutation_metadata['data']['cases'][0]['samples'][0]['sample_id']
# mutation_metadata['data']['cases'][0]['samples'][0]['portions'][0]['analytes'][0]['aliquots'][0]['aliquot_id']

UUID: 0019c951-16c5-48d0-85c8-58d96b12d330
{'samples': [{'portions': [{'analytes': [{'aliquots': [{'aliquot_quantity': 4.27, 'amount': None, 'aliquot_id': 'eddd285e-a946-4e4e-a21d-6e84ed397594', 'source_center': '23', 'analyte_type': None, 'updated_datetime': '2022-04-28T23:44:19.603620-05:00', 'submitter_id': 'TCGA-D8-A1XO-01A-11R-A14M-07', 'concentration': 0.16, 'state': 'released', 'aliquot_volume': 26.7, 'analyte_type_id': None, 'created_datetime': None}]}]}]}]}


In [21]:
def extract_aliquot_ids(metadata):
    aliquot_ids = []
    try:
        for sample in metadata['data']['cases'][0]['samples']:
            for portion in sample.get('portions', []):
                for analyte in portion.get('analytes', []):
                    for aliquot in analyte.get('aliquots', []):
                        aliquot_ids.append(aliquot['aliquot_id'])
    except Exception as e:
        print("Error while parsing aliquot IDs:", e)
    return aliquot_ids

In [25]:
print('Mutation aliquots:', extract_aliquot_ids(mutation_metadata))
print('Expression aliquots:', extract_aliquot_ids(expression_metadata))

Mutation aliquots: ['b3568259-c63c-4eb1-bbc7-af711ddd33db', '17ba8cdb-e35b-4496-a787-d1a7ee7d4a1e']
Expression aliquots: ['eddd285e-a946-4e4e-a21d-6e84ed397594']


Unzip mutation files

In [25]:
import gzip
import os
import re

In [38]:
mutation_aliquot_ids = []
for file_id in os.listdir(mutation_dir):
    for filename in os.listdir(mutation_dir+file_id):
        if re.match(r"(.+)\.wxs\.aliquot_ensemble_masked\.maf\.gz", filename):
            name = mutation_dir + file_id + "/" + filename
            with gzip.open(name, 'rb') as f:
                content = f.read().decode('utf-8')
                match = re.search(r"^#tumor\.aliquot (.+)$", content, re.MULTILINE)
                if match:
                    aliquot_id = str(match.group(1))
                    mutation_aliquot_ids.append(aliquot_id)

print(len(mutation_aliquot_ids))
mutation_aliquot_ids = set(mutation_aliquot_ids)
print(len(mutation_aliquot_ids))


992
992


Inspect names

In [ ]:
import os
import re

In [11]:
expression_case_ids = os.listdir(expression_dir)

In [ ]:
def extract_uuids(dir, ptrn):
    res = []
    for sample_id in os.listdir(dir):
        for filename in os.listdir(dir+sample_id):
            match = re.search(ptrn, filename, re.DOTALL)
            if match:
                res.append(match.group(1)) 
    return res

expression_uuids = set(extract_uuids(expression_dir, r"^(.+)\.rna_seq\.augmented_star_gene_counts"))
mutation_uuids = set(extract_uuids(mutation_dir, r"(.+)\.wxs\.aliquot_ensemble_masked\.maf\.gz"))

uuids = expression_uuids.intersection(mutation_uuids)
print('Number of expression samples:', len(expression_uuids))
print('Number of mutation samples:', len(mutation_uuids))
print('Number of shared uuids:', len(uuids))

Number of expression samples: 1231
Number of mutation samples: 992
Number of shared uuids: 0


In [44]:
expression_uuids & mutation_aliquot_ids

set()

In [59]:
expression_uuids & mutation_entity_ids

set()

In [61]:
expression_case_ids & mutation_aliquot_ids

set()

In [62]:
expression_case_ids & mutation_entity_ids

set()

In [1]:
sample_id = 'f6392b6f-24d9-4768-8269-c7c647f8ff8c'
uuid = '0e062eef-29b8-4622-afd9-0d3f0d1c37d6'

In [4]:
file_path = f'mrna_expression_data/{sample_id}/{uuid}.rna_seq.augmented_star_gene_counts.tsv'

In [5]:
import pandas as pd
data = pd.read_csv(file_path, sep='\t', skiprows=1)

In [ ]:
data.head()

,gene_id,gene_name,gene_type,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded
0,N_unmapped,NaN,NaN,7621550,7621550,7621550,NaN,NaN,NaN
1,N_multimapping,NaN,NaN,4820670,4820670,4820670,NaN,NaN,NaN
2,N_noFeature,NaN,NaN,2249322,32825579,32790945,NaN,NaN,NaN
3,N_ambiguous,NaN,NaN,6332806,1607967,1607457,NaN,NaN,NaN
4,ENSG00000000003.15,TSPAN6,protein_coding,2269,1119,1150,28.866,8.9476,8.6757


In [ ]:
data['fpkm_uq_unstranded']

0           NaN
1           NaN
2           NaN
3           NaN
4        8.6757
          ...  
60659    0.0000
60660    1.3629
60661    0.0000
60662    0.0180
60663    0.1549
Name: fpkm_uq_unstranded, Length: 60664, dtype: float64

### Mutation data

In [ ]:
import gzip
import os
import pickle
import re
import requests

Query the database to get the case id for the mutation data

In [4]:
def lookup_case_metadata(file_uuid):
    url = f"https://api.gdc.cancer.gov/files/{file_uuid}?expand=cases"
    response = requests.get(url)
    return response.json()

In [5]:
def lookup_aliquots_metadata(file_uuid):
    url = f"https://api.gdc.cancer.gov/files/{file_uuid}?expand=cases.samples.portions.analytes.aliquots"
    response = requests.get(url)
    return response.json()

In [49]:
eid_to_mutations = {}
for case_id in os.listdir(mutation_dir):
    try:
        annotations_df = pd.read_csv(mutation_dir+case_id+'/annotations.txt', sep='\t')
        eid_to_mutations[annotations_df['entity_id'].iloc[0]] = mutation_dir+case_id
    except:
        print(f'Missing annotation: {case_id}')

print(len(mutation_entity_ids))

Missing annotation: 1e171ca7-9c44-4dc9-8908-b35b40e3464e
Missing annotation: 23c0af0c-102c-40e6-8e93-7ec13bc032b9
Missing annotation: 2e9ac636-1882-4d71-b672-c057d2730f03
Missing annotation: 56d15f8e-3788-4715-9ec8-853a107679a4
Missing annotation: 7a7dcf94-b3d7-4516-9736-db8fdb7b451a
Missing annotation: 803decea-3a3f-42bf-8122-328bc32aa850
Missing annotation: c1a21a51-b8b7-453d-932e-2eacbaed8c2d
Missing annotation: cb5fd63b-da6d-4016-b150-2a0682d00db2
Missing annotation: e0e31082-4736-4917-a72b-9bf455239984
Missing annotation: f3508f1c-34c8-4e72-87b3-0163273b0393
Missing annotation: f90b3b34-16b3-435c-bbf1-82a1450ec7f0
981


In [47]:
def get_case_id(metadata):
    try:
        return metadata['data']['cases'][0]['case_id']
    except (KeyError, IndexError):
        return None

In [59]:
cid_to_mutations = {}
for eid, mutation in eid_to_mutations.items():
    cid = get_case_id(lookup_case_metadata(eid))
    if cid:
        cid_to_mutations[cid] = mutation
    else:
        print(f'No cid found for {eid}')

No cid found for dfefb76a-ec6b-4cd2-9d45-2e1e4befc7ea
No cid found for e6f527a2-38eb-457e-bb39-3c5b9c0bcbec
No cid found for 74f3a478-527e-4d49-9c3f-29f353e1fb6c
No cid found for c1442467-d618-435f-8792-b1a18f696f1a
No cid found for 6bc4c56d-aac5-49a2-9913-9538d06d898f
No cid found for 7291657b-2a8c-467d-a763-3de577b6401b
No cid found for 6f959c92-b79f-4f84-9ee5-d07d3212d52d
No cid found for 36228261-b2db-4465-869a-672916782b71
No cid found for b3354b72-2637-4edc-9a95-e55801d3ac13
No cid found for 2779fa01-ac93-4e80-a997-3385f72172c3
No cid found for 024b6e54-dc95-457d-a70d-9db56806159f
No cid found for 33fe5833-d7a0-43d7-a03d-56985ea448a6
No cid found for 44e34ec8-b65b-451c-aea9-89a6537bc689
No cid found for 80f3f48a-cc21-415c-b181-c77e7ba1c563
No cid found for 91ac9c48-a751-4e9a-96c3-558bb07c2754
No cid found for e27e9375-d153-4d0d-80ea-19c0f58c6c60


In [ ]:
# Save the mapping
import pickle
with open('cid_to_mutations.pkl', 'wb') as f:
    pickle.dump(cid_to_mutations, f)

In [8]:
# Load the mapping
import pickle
with open('cid_to_mutations.pkl', 'rb') as f:
    cid_to_mutations = pickle.load(f)

### Expression data

Lookup the expression file aliquot id

In [31]:
def extract_aliquot_ids(metadata):
    aliquot_ids = []
    try:
        for sample in metadata['data']['cases'][0]['samples']:
            for portion in sample.get('portions', []):
                for analyte in portion.get('analytes', []):
                    for aliquot in analyte.get('aliquots', []):
                        aliquot_ids.append(aliquot['aliquot_id'])
    except Exception as e:
        print("Error while parsing aliquot IDs:", e)
    return aliquot_ids

In [40]:
def extract_uuids(dir, ptrn):
    res = []
    for sample_id in os.listdir(dir):
        for filename in os.listdir(dir+sample_id):
            match = re.search(ptrn, filename, re.DOTALL)
            if match:
                res.append(match.group(1)) 
    return res

expression_uuids = set(extract_uuids(expression_dir, r"^(.+)\.rna_seq\.augmented_star_gene_counts"))

In [ ]:
# For each case id in the expression dataset
for id in os.listdir(expression_dir):
    # Look up the aliquot ids associated with this case id - should only be one
    aliquots = extract_aliquot_ids(lookup_file_metadata(id))
    print(aliquots)
    for aliquot_id in aliquots:
        if aliquot_id in aliquot_to_files:
            # Get the expression filename - should only be one
            expression_file = [file for file in os.listdir(expression_dir+id) if file.endswith('.tsv')][0]
            # Add it to the files
            aliquot_to_files[aliquot_id].append(expression_file)
            print('Aliquot not found in map:', aliquot_id)
        else:
            print('Aliquot not found in map:', aliquot_id)

Look up expression case ids

In [69]:
cid_to_expressions = {}
for expr_id in os.listdir(expression_dir):
    cid = get_case_id(lookup_case_metadata(expr_id))
    if cid:
        cid_to_expressions[cid] = expression_dir+expr_id
    else:
        print(f'cid not found for {expr_id}')

In [ ]:
# Save the mapping
import pickle
with open('cid_to_expressions.pkl', 'wb') as f:
    pickle.dump(cid_to_expressions, f)

In [9]:
# Load the mapping
with open('cid_to_expressions.pkl', 'rb') as f:
    cid_to_expressions = pickle.load(f)

In [10]:
cids = list(set(cid_to_expressions) & set(cid_to_mutations))

In [78]:
print(f'{len(cids)} shared cases')

939 shared cases


### Build the dataset

In [11]:
import gzip
import numpy as np
import os
import pandas as pd

In [12]:
#
# Create the gene ordering.
#   The gene ordering ensures all expression counts are ordered the same way in the feature vectors
#

gene_ordering =  set()

for cid in cids:
    #
    # Load the expression dataframe
    #
    expr_case_dir = cid_to_expressions[cid]
    tsv_file = [file for file in os.listdir(expr_case_dir) if file.endswith('.tsv')][0]
    expr_df = pd.read_csv(expr_case_dir+'/'+tsv_file, sep='\t', skiprows=1)
    #
    # Pull the gene names from the expr dataframe
    #
    gene_ids = set(expr_df.gene_id[~expr_df.gene_id.isna() & ~expr_df.gene_name.isna()])
    #
    # Check if this expression dataframe contains any unqiue gene names
    #
    diff = gene_ids - gene_ordering
    for new_gene in diff:
        gene_ordering.add(new_gene)

#
# Convert the gene_ordering to a list to enforce an ordering
#
gene_ordering = list(gene_ordering)

In [13]:
print('Number of genes in expression dataset:', len(gene_ordering))

Number of genes in expression dataset: 60660


In [14]:
# List of non-silent mutations to classify
mutation_types = [
    'Missense_Mutation',     # alters one amino acid
    'Nonsense_Mutation',     # early stop codone
    'Frame_Shift_Del',       # shifts 'reading frame' after deletion
    'Frame_Shift_Ins',       # shifts 'reading frame' after insertion
    'Splice_Site',           # dirupted splicing
    'Nonstop_Mutation',      # missing stop codone
    'Translation_Start_Site' # mutated start codone
]

In [15]:
X = [] # Feature matrix - gene expressions
y = [] # Label vectory - binary, whether P25 gene is mutated

#
# For each case id that is shared between the expression and mutation datasets...
#
for cid in cids:
    #
    # --> Load the expression dataframe
    #
    expr_case_dir = cid_to_expressions[cid]
    tsv_file = [file for file in os.listdir(expr_case_dir) if file.endswith('.tsv')][0]
    expr_df = pd.read_csv(expr_case_dir+'/'+tsv_file, sep='\t', skiprows=1)
    #
    # Sort the rows by gene ordering
    #
    expr_df = expr_df[expr_df['gene_id'].isin(gene_ordering)] # Exclude rows missing from gene ordering
    expr_df = expr_df.set_index('gene_id').loc[gene_ordering]  # Ensure consistent order
    #
    # Get the fpkm_uq_unstranded column as the feature vector
    #
    x = expr_df['fpkm_uq_unstranded'].fillna(0.).values
    #
    # Add this feature vector to the feature matrix
    #
    X.append(x)
    #
    # --> Load the mutation dataframe
    #
    mut_case_dir = cid_to_mutations[cid]
    mut_file = [file for file in os.listdir(mut_case_dir) if file.endswith('.maf.gz')][0]
    with gzip.open(mut_case_dir+'/'+mut_file, 'rb') as f:
        mut_df = pd.read_csv(f, sep='\t', skiprows=7)
    #
    # Isolate the TP53 gene in the mutation file
    #
    tp53 = mut_df[mut_df['Hugo_Symbol'] == 'TP53']
    #
    # Check for non-silent mutations
    #
    mutated = False
    if not tp53.empty:
        mutations = tp53[
            tp53['Variant_Classification'].isin(mutation_types)
        ]
        mutated = not mutations.empty
    #
    # Add the mutated boolean to the label vector
    #
    y.append(mutated)


C:\Users\SBC98\AppData\Local\Temp\ipykernel_25736\331707311.py:33: DtypeWarning: Columns (114,120) have mixed types. Specify dtype option on import or set low_memory=False.
  mut_df = pd.read_csv(f, sep='\t', skiprows=7)


In [16]:
#
# To torch
#
import torch
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

C:\Users\SBC98\AppData\Local\Temp\ipykernel_25736\1330128375.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  X = torch.tensor(X, dtype=torch.float32)


In [17]:
#
# Save
#
torch.save({'features': X, 'labels': y}, 'data.pt')